In [1]:
from pathlib import Path
import torch
import torch.nn as nn
from torchtext.datasets import SST2
from torch.utils.data import DataLoader
from newModel import make_model
from torch.utils.data import DataLoader, Dataset
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from google_drive_downloader import GoogleDriveDownloader as gdd
import torch.optim as optim
from tqdm import tqdm, tqdm_notebook

/Users/vale/opt/anaconda3/envs/torch_env/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
DATA_PATH = '../data/imdb_reviews.csv'
if not Path(DATA_PATH).is_file():
    gdd.download_file_from_google_drive(
        file_id='1zfM5E6HvKIe7f3rEt1V2gBpw5QOSSKQz',
        dest_path=DATA_PATH,
    )

In [4]:
class Sequences(Dataset):
    def __init__(self, path):
        df = pd.read_csv(path)
        self.vectorizer = CountVectorizer(stop_words='english', max_df=0.99, min_df=0.005)
        self.sequences = self.vectorizer.fit_transform(df.review.tolist())
        self.labels = df.label.tolist()
        self.token2idx = self.vectorizer.vocabulary_
        self.idx2token = {idx: token for token, idx in self.token2idx.items()}
        
    def __getitem__(self, i):
        return self.sequences[i, :].toarray(), self.labels[i]
    
    def __len__(self):
        return self.sequences.shape[0]

In [5]:
dataset = Sequences(DATA_PATH)
train_loader = DataLoader(dataset, batch_size=1)

print(dataset[5][0].shape)

(1, 3028)


In [6]:
feature, label = next(iter(train_loader))
print(feature[0][0], label)
src = torch.LongTensor(feature[0][0])
print(src)
print(src.shape)
print(feature.shape)

tensor([0, 0, 0,  ..., 0, 0, 0]) tensor([0])
tensor([0, 0, 0,  ..., 0, 0, 0])
torch.Size([3028])
torch.Size([1, 1, 3028])


In [7]:
len(dataset.token2idx)

3028

In [35]:
model = make_model(len(dataset.token2idx)+1, N=2, d_model=32, d_ff=64)
model = make_model(src2.max() + 1, N=2, d_model=32, d_ff=64)

In [54]:
src = torch.LongTensor([[11, 2, 3, 4, 5, 6, 7, 8, 9, 20, 10]])
print(src.max())
print(src.shape)

model = make_model(21, N=2, d_model=32, d_ff=64)
model.eval()


ys = model(src)
print(ys)


tensor(20)
torch.Size([1, 11])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x352 and 320x2)

In [44]:
feature, label = next(iter(train_loader))
print(feature[0][0], label)
src2 = feature[0]
print(src.shape)

model = make_model(src.max()+1, N=2, d_model=32, d_ff=64)
model.eval()


ys = model(src)
print(ys)

tensor([0, 0, 0,  ..., 0, 0, 0]) tensor([0])
torch.Size([1, 10])
1


In [36]:
model.eval()
src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 11]])
print(src.max())
print(src.shape)
feature, label = next(iter(train_loader))
print(feature[0][0], label)
src2 = feature[0]
print(src.shape)


ys = model(src2)
print(src2.shape)
print(ys)

tensor(11)
torch.Size([1, 10])
tensor([0, 0, 0,  ..., 0, 0, 0]) tensor([0])
torch.Size([1, 10])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x96896 and 320x2)

In [37]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
model.train()
train_losses = []
for epoch in range(25):
    progress_bar = tqdm_notebook(train_loader, leave=False)
    losses = []
    total = 0
    for inputs, target in progress_bar:
        inputs = inputs.type(torch.FloatTensor).to(device)
        model.zero_grad()
        print(inputs.shape)
        print(inputs[0])
        print(target.shape)

        output = model(inputs)
        loss = criterion(output.squeeze(), target.float())
        
        loss.backward()
              
        nn.utils.clip_grad_norm_(model.parameters(), 3)

        optimizer.step()
        
        progress_bar.set_description(f'Loss: {loss.item():.3f}')
        
        losses.append(loss.item())
        total += 1
    
    epoch_loss = sum(losses) / total
    train_losses.append(epoch_loss)
        
    tqdm.write(f'Epoch #{epoch + 1}\tTrain Loss: {epoch_loss:.3f}')

/var/folders/bn/8vrffm796w3c4w6p5lkp0l4c0000gn/T/ipykernel_50903/1755946833.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progress_bar = tqdm_notebook(train_loader, leave=False)


  0%|          | 0/16 [00:00<?, ?it/s]

torch.Size([4096, 1, 3028])
tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([4096])


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)